In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import os

In [2]:
# This shows how to save/restore your model (trained variables).
# To see how it works, please stop this program during training and resart.
# This network is the same as 3_net.py

def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

def model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden): # this network is the same as the previous one except with an extra hidden layer + dropout
    X = tf.nn.dropout(X, p_keep_input)
    h = tf.nn.relu(tf.matmul(X, w_h))

    h = tf.nn.dropout(h, p_keep_hidden)
    h2 = tf.nn.relu(tf.matmul(h, w_h2))

    h2 = tf.nn.dropout(h2, p_keep_hidden)

    return tf.matmul(h2, w_o)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)



In [4]:
ckpt_dir = "./ckpt_dir"
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

global_step = tf.Variable(0, name='global_step', trainable=False)

# Call this after declaring all tf.Variables.
saver = tf.train.Saver()

# This variable won't be stored, since it is declared after tf.train.Saver()
non_storable_variable = tf.Variable(777)

In [4]:
saver

In [5]:
sess = tf.Session()

In [6]:
#tf.global_variables_initializer().run()
sess.run(tf.global_variables_initializer())

In [7]:
!ls -al

total 600
drwxr-xr-x   6 root root   4096 Mar  4 15:10 .
drwxrwxr-x 204 root root  12288 Mar  4 14:01 ..
-rw-r--r--   1 root root   1575 Mar  4 14:01 00_multiply.ipynb
-rwxr-xr-x   1 root root    500 Mar  4 14:01 00_multiply.py
-rw-r--r--   1 root root   2550 Mar  4 14:01 01_linear_regression.ipynb
-rwxr-xr-x   1 root root   1093 Mar  4 14:01 01_linear_regression.py
-rw-r--r--   1 root root   3009 Mar  4 14:01 02_logistic_regression.ipynb
-rwxr-xr-x   1 root root   1610 Mar  4 14:01 02_logistic_regression.py
-rw-r--r--   1 root root   2917 Mar  4 14:01 03_net.ipynb
-rwxr-xr-x   1 root root   1500 Mar  4 14:01 03_net.py
-rw-r--r--   1 root root   3504 Mar  4 14:01 04_modern_net.ipynb
-rwxr-xr-x   1 root root   1953 Mar  4 14:01 04_modern_net.py
-rw-r--r--   1 root root   5428 Mar  4 14:01 05_convolutional_net.ipynb
-rwxr-xr-x   1 root root   3540 Mar  4 14:01 05_convolutional_net.py
-rw-r--r--   1 root root 386323 Mar  4 14:01 06_autoencoder.ipynb
-rw-r--r--   1 root root   3135 Mar  4 

In [5]:
# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    ckpt = tf.train.get_checkpoint_state(ckpt_dir)
    if ckpt and ckpt.model_checkpoint_path:
        print(ckpt.model_checkpoint_path)
        saver.restore(sess, ckpt.model_checkpoint_path) # restore all variables

    start = global_step.eval() # get last global_step
    print("Start from:", start)

    for i in range(start, 10):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_input: 0.8, p_keep_hidden: 0.5})

        global_step.assign(i).eval() # set and update(eval) global_step with index, i
        saver.save(sess, ckpt_dir + "/model.ckpt", global_step=global_step)
        print(i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX, Y: teY,
                                                         p_keep_input: 1.0,
                                                         p_keep_hidden: 1.0})))

Start from: 0
0 0.9291
1 0.9613
2 0.9687
3 0.9742
4 0.9763
5 0.9771
6 0.979
7 0.9796
8 0.9789
9 0.9793
10 0.9805
11 0.9807
12 0.9799


KeyboardInterrupt: 

In [6]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(784, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(625, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(625, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_3:0' shape=() dtype=int32_ref>]

In [8]:
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[<tf.Variable 'Variable:0' shape=(784, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_1:0' shape=(625, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_2:0' shape=(625, 10) dtype=float32_ref>,
 <tf.Variable 'Variable/RMSProp:0' shape=(784, 625) dtype=float32_ref>,
 <tf.Variable 'Variable/RMSProp_1:0' shape=(784, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_1/RMSProp:0' shape=(625, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_1/RMSProp_1:0' shape=(625, 625) dtype=float32_ref>,
 <tf.Variable 'Variable_2/RMSProp:0' shape=(625, 10) dtype=float32_ref>,
 <tf.Variable 'Variable_2/RMSProp_1:0' shape=(625, 10) dtype=float32_ref>,
 <tf.Variable 'global_step:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Variable_3:0' shape=() dtype=int32_ref>]

In [10]:
# reset the graph (erase all variables)
tf.reset_default_graph()

In [12]:
# it should return a blank list
tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

[]

In [13]:
# restore from checkpoint
# 1. construct the network model
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, 625])
w_h2 = init_weights([625, 625])
w_o = init_weights([625, 10])

p_keep_input = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w_h, w_h2, w_o, p_keep_input, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

In [18]:
# 2. create a new session and initialize all variables
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [21]:
# 3. restore weights from checkpoint,  透過讀取 "checkpoint" txt file來知道哪個是the latest checkpoint
ckpt = tf.train.get_checkpoint_state(os.path.dirname('./ckpt_dir/checkpoint'))

In [22]:
# we know that the latest checkpoint is model.ckpt-12
ckpt

model_checkpoint_path: "./ckpt_dir/model.ckpt-12"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-8"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-9"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-10"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-11"
all_model_checkpoint_paths: "./ckpt_dir/model.ckpt-12"

In [23]:
# by ckpt.model_checkpoint_path
ckpt.model_checkpoint_path

'./ckpt_dir/model.ckpt-12'

In [25]:
# If the graph had been reset the graph (erase all variables), remember to create saver node
saver = tf.train.Saver()

In [26]:
# 4. use saver.restore to restore the weights
if ckpt and ckpt.model_checkpoint_path:
     saver.restore(sess, ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from ./ckpt_dir/model.ckpt-12
